In [ ]:
import sys
sys.path.append('/data_nfs/og86asub/netmap/netmap-evaluation/')

import scanpy as sc
import time 

from netmap.src.utils.misc import write_config

from netmap.src.model.negbinautoencoder import *
import scanpy as sc

from sklearn.model_selection import train_test_split
import time
from captum.attr import GradientShap, LRP
from netmap.src.model.inferrence_simple import *
from netmap.src.utils.data_utils import attribution_to_anndata
from netmap.src.model.pipeline import *
import numpy as np


from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import HDBSCAN

import os.path as op
import os

import anndata as ad
from statsmodels.stats.nonparametric import rank_compare_2indep

import numpy as np
import pandas as pd
import scipy.sparse as scs
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import HDBSCAN
from captum.attr import *
import pingouin as pingu
import torch
import torch.nn as nn




    

def create_model_zoo(data_tensor, n_models = 4, n_epochs = 500):
    model_zoo = []
    for _ in range(n_models):

        data_train2, data_test2 = train_test_split(data_tensor,test_size=0.01, shuffle=True)

        trained_model2 = NegativeBinomialAutoencoder(input_dim=data_tensor.shape[1], latent_dim=10, dropout_rate = 0.02)
        trained_model2 = trained_model2.cuda()

        optimizer2 = torch.optim.Adam(trained_model2.parameters(), lr=1e-4)

        trained_model2 = train_autoencoder(
                trained_model2,
                data_train2.cuda(),
                optimizer2,
                num_epochs=n_epochs

            )
        model_zoo.append(trained_model2)
    return model_zoo


def create_model_zoo(data_tensor, n_models = 4, n_epochs = 500):
    model_zoo = []
    for _ in range(n_models):

        data_train2, data_test2 = train_test_split(data_tensor,test_size=0.01, shuffle=True)

        trained_model2 = ZINBAutoencoder(input_dim=data_tensor.shape[1], latent_dim=10, dropout_rate = 0.02)
        trained_model2 = trained_model2.cuda()

        optimizer2 = torch.optim.Adam(trained_model2.parameters(), lr=1e-4)

        trained_model2 = train_autoencoder(
                trained_model2,
                data_train2.cuda(),
                optimizer2,
                num_epochs=n_epochs

            )
        model_zoo.append(trained_model2)
    return model_zoo




def set_latent_true(model_zoo):
    for mo in model_zoo:
        mo.forward_mu_only = False
        mo.forward_theta_only = False
        mo.latent_only = True
    return model_zoo


def set_all_false(model_zoo):
    for mo in model_zoo:
        mo.forward_mu_only = False
        mo.forward_theta_only = False
        mo.latent_only = False
    return model_zoo

def shuffle_each_column_independently(tensor):
    """
    Shuffles each column of a 2D PyTorch tensor independently.

    Args:
        tensor (torch.Tensor): The input tensor.

    Returns:
        torch.Tensor: A new tensor with each of its columns independently shuffled.
    """
    if tensor.dim() != 2:
        raise ValueError("Input tensor must be 2-dimensional to shuffle columns.")

    # Create an empty tensor of the same size to store the shuffled columns
    shuffled_tensor = torch.empty_like(tensor)

    # Iterate through each column, shuffle it, and place it in the new tensor
    for i in range(tensor.size(1)):
        column = tensor[:, i]
        idx = torch.randperm(column.nelement())
        shuffled_tensor[:, i] = column[idx]

    return shuffled_tensor


def attribution_one_target( 
        target_gene,
        lrp_model,
        input_data,
        background,
        xai_type='lrp-like',
        randomize_background = False):
    
    attributions_list = []
    for m in range(len(lrp_model)):
        # Randomize backgorund for each round
        if randomize_background:
            background = shuffle_each_column_independently(background)

        model = lrp_model[m]
        #for _ in range(num_iterations):
        if xai_type == 'lrp-like':
            #print(input_data)
            #print(target_gene)
            attribution = model.attribute(input_data, target=target_gene)
                
        elif xai_type == 'shap-like':
            attribution = model.attribute(input_data, baselines = background, target = target_gene)

        attributions_list.append(attribution.detach().cpu().numpy())
    return attributions_list

def get_differential_edges(attribution_anndata, percentile = 10):
    genelist = []
    if len(np.unique(attribution_anndata.obs['leiden']))>1 :
        for cat in np.unique(attribution_anndata.obs['leiden']):
            statisi =rank_compare_2indep(x1=attribution_anndata.X[attribution_anndata.obs['leiden']==cat], x2= attribution_anndata.X[attribution_anndata.obs['leiden']!=cat])
            sig_and_high = np.where((statisi.pvalue<(0.01/(attribution_anndata.X.shape[1]*attribution_anndata.X.shape[1])))  & (statisi.prob1>= 0.9))
            genelist = genelist+ list(sig_and_high[0])

    else:
        # FALLBACk
        m = np.abs(attribution_anndata.X).mean(axis=0)
        # Get the indices of genes in the top 10%
        top_10_percent_indices = np.where(m > np.percentile(m, 100-percentile))[0]

        # Get the indices of genes in the bottom 10%
        bottom_10_percent_indices = np.where(m < np.percentile(m, percentile))[0]

        # Combine the two arrays of indices and sort them
        genelist = np.unique(np.sort(
            np.concatenate((top_10_percent_indices, bottom_10_percent_indices))
        ))
    return genelist

def get_percentile_edges(attribution_anndata, percentile = 10):
    # FALLBACk
    m = attribution_anndata.X.mean(axis=0)
    # Get the indices of genes in the top 10%
    top_10_percent_indices = np.where(m > np.percentile(m, 100-percentile))[0]

    # Get the indices of genes in the bottom 10%
    bottom_10_percent_indices = np.where(m < np.percentile(m, percentile))[0]

    # Combine the two arrays of indices and sort them
    genelist = np.unique(np.sort(
        np.concatenate((top_10_percent_indices, bottom_10_percent_indices))
    ))
    return genelist

def get_edges(attribution_anndata, use_differential=False, percentile = 10):
    if use_differential:
        return get_differential_edges(attribution_anndata, percentile=percentile)
    else:
        return get_percentile_edges(attribution_anndata, percentile=percentile)
    
def get_explainer(model, explainer_type, raw=False):
    if explainer_type in ['GuidedBackprop', 'Deconvolution']:
        explainer_mode = 'lrp-like'
    else:
        explainer_mode = 'shap-like'
    
        
    if explainer_type == 'GuidedBackprop': #fast
        explainer = GuidedBackprop(model)
    elif explainer_type == 'GradientShap': #fast
        if raw:
            explainer = GradientShap(model, multiply_by_inputs=False)
        else:
            explainer = GradientShap(model, multiply_by_inputs=True)

    elif explainer_type == 'Deconvolution': #fast
        explainer = Deconvolution(model)
    else:
        raise ValueError('no such method')
        
    return explainer, explainer_mode

def compute_correlation_metric(data, cor_type):
    # Compute gene correlation measure
    #  'pingouin.pcorr', 'np.cov', 'np.corcoeff'
    if cor_type ==  'pingouin.pcorr':
        cov = pingu.pcorr(pd.DataFrame(data))
    elif cor_type == 'np.cov':
        cov = np.cov(data.T)
    elif cor_type == 'np.corrcoeff':
        cov = np.corrcoef(data.T)
    elif cor_type == 'None':
        cov = 1
    else: 
        cov = 1
    return cov

def aggregate_attributions(attributions, strategy = 'mean'):
    if strategy == 'mean':
        return np.mean(attributions, axis = 0)
    elif strategy == 'sum':
        return np.sum(attributions, axis = 0)
    elif strategy == 'median':
        return np.median(attributions, axis = 0)
    else:
        # Default to mean aggregation
        return np.mean(attributions, axis = 0)
    

    
def wrapper(models, data_train_full_tensor, gene_names, config):

    data = data_train_full_tensor.detach().cpu().numpy()
    tms = []
    name_list = []
    target_names = []
    
    
    
    
    ings = {}
    for trained_model in models:        
        trained_model.forward_mu_only = True
        explainer, xai_type = get_explainer(trained_model, config.xai_method, config.raw_attribution)
        tms.append(explainer)

    attributions = []
    ## ATTRIBUTIONS
    for g in tqdm(range(data_train_full_tensor.shape[1])):
    #for g in range(2):

        attributions_list = attribution_one_target(
            g,
            tms,
            data_train_full_tensor,
            data_train_full_tensor,
            xai_type=xai_type,
            randomize_background = True)
        attributions.append(attributions_list)

    

    ## AGGREGATION: REPLACE LIST BY AGGREGATED DATA
    for i in range(len(attributions)):
        # CURRENTLY MEAN
        attributions[i] = aggregate_attributions(attributions[i], strategy=config.aggregation_strategy )
    
    print(attributions)
    ## PENALIZE:
    if config.penalty != 'None':
        penalty_matrix = compute_correlation_metric(data, cor_type=config.penalty)
        for i in range(len(attributions)):
            # CURRENTLY MEAN
            attributions[i] = np.dot(attributions[i], (1-penalty_matrix))

    print(attributions)
    
    ## CLUSTERING: CLUSTER EACH TARGET INDVIDUALLY
    for i in range(len(attributions)):
 
        attributions[i] = ad.AnnData(attributions[i])
        print(attributions[i])
        sc.pp.scale(attributions[i])
        try:
            sc.pp.pca(attributions[i],n_comps=50)
        except:
            try:
                sc.pp.pca(attributions[i],n_comps=50 )
            except:
                continue
            
        sc.pp.neighbors(attributions[i], n_neighbors=15)
        sc.tl.leiden(attributions[i], resolution=0.1)

        clusterings[f'T_{gene_names[i]}'] = np.array(attributions[i].obs['leiden'])

    
    #EDGE SELECTION:
    for i in range(len(attributions)):
        edge_indices = get_edges(attributions[i], use_differential=config.use_differential, percentile=config.percentile)
        name_list = name_list + list(gene_names[edge_indices])
        target_names = target_names+[gene_names[i]]* len(edge_indices)
        attributions[i] = attributions[i][:,edge_indices].X

    attributions = np.hstack(attributions)
    
    index_list = [f"{s}_{t}" for (s, t) in zip(name_list, target_names)]
    cou = pd.DataFrame({'index': index_list, 'source':name_list, 'target':target_names})
    cou = cou.set_index('index')

    clusterings = pd.DataFrame(clusterings)

    grn_adata = attribution_to_anndata(attributions, var=cou, obs = clusterings)

    return grn_adata

def run_netmap(config, dataset_config):

    print('Version 2')
    start_total = time.monotonic()
    
    ## Load config and setup outputs
    os.makedirs(config.output_directory, exist_ok=True)
    sc.settings.figdir = config.output_directory
    config.write_yaml(yaml_file=op.join(config.output_directory, 'config.yaml'))

    ## load data
    adata = sc.read_h5ad(config.input_data)
    

    ## Get the data matrix from the CustumAnndata obeject

    gene_names = np.array(adata.var.index)
    model_start = time.monotonic()

    if config.layer == 'counts':
        data_tensor = adata.layers['counts']
    else:
        data_tensor = adata.X

    if scs.issparse(data_tensor):
        data_tensor = torch.tensor(data_tensor.todense(), dtype=torch.float32)
    else:
        data_tensor = torch.tensor(data_tensor, dtype=torch.float32)


    print(data_tensor.shape)

    model_zoo = create_model_zoo(data_tensor, n_models=config.n_models, n_epochs=500)
    grn_adata = wrapper(model_zoo, data_tensor.cuda(), gene_names, config)

    adob = adata.obs.reset_index()
    grn_adata.obs['cell_id'] = np.array(adob['cell_id'])
    grn_adata.obs['grn'] = np.array(adob['grn'])

    
    model_elapsed = time.monotonic()-model_start
    grn_adata.write_h5ad(op.join(config.output_directory,config.adata_filename))

    time_elapsed_total = time.monotonic()-start_total


    res = {'time_elapsed_total': time_elapsed_total, 'time_elapsed_netmap': model_elapsed} 
    write_config(res, file=op.join(config.output_directory, 'results.yaml'))



In [ ]:



sys.path.append('/data_nfs/og86asub/netmap/netmap-evaluation/')
from netmap.src.utils.data_utils import *
from netmap.src.utils.tf_utils import *
from netmap.src.utils.netmap_config import NetmapConfig
from netmap.src.model.negbinautoencoder import *
from netmap.src.model.negbinautoencoder import train_autoencoder
from netmap.src.model.inferrence_simple import *
from netmap.src.model.pipeline import *

from src.data_simulation.data_simulation_config import DataSimulationConfig


import yaml
def read_config(file):
    with open(file, "r") as f:
        config = yaml.safe_load(f)
    return config

import os.path as op

#config = NetmapConfig.read_yaml("/data_nfs/og86asub/netmap/netmap-evaluation/results/configurations/netmap/config/perturb_seq/")
dada = "/data_nfs/og86asub/netmap/netmap-evaluation/results/configurations/data_simulation/config_easy/net_172_54892_net_131_54992_net_158_55084.config.yaml"
dataset_config = read_config("/data_nfs/og86asub/netmap/netmap-evaluation/results/configurations/data_simulation/config_easy/net_172_54892_net_131_54992_net_158_55084.config.yaml")

nets = [pd.read_csv(op.join("/data_nfs/og86asub/netmap/netmap-evaluation/data/clustered_network/", filename), sep='\t') for filename in dataset_config['edgelist']]
common = [pd.read_csv(op.join("/data_nfs/og86asub/netmap/netmap-evaluation/data/clustered_network/", filename), sep='\t') for filename in dataset_config['common_edges']]

    
config = NetmapConfig.read_yaml('/data_nfs/og86asub/netmap/netmap-evaluation/results/netmap/config_22/config_easy/net_172_54892_net_131_54992_net_158_55084/config.yaml')
dataset_config = DataSimulationConfig.read_yaml(dada)



start_total = time.monotonic()

## Load config and setup outputs
os.makedirs(config.output_directory, exist_ok=True)
sc.settings.figdir = config.output_directory
config.write_yaml(yaml_file=op.join(config.output_directory, 'config.yaml'))

## load data
adata = sc.read_h5ad(config.input_data)


## Get the data matrix from the CustumAnndata obeject
gene_names = np.array(adata.var.index)
model_start = time.monotonic()

if config.layer == 'counts':
    data_tensor = adata.layers['counts']
else:
    data_tensor = adata.X

if scs.issparse(data_tensor):
    data_tensor = torch.tensor(data_tensor.todense(), dtype=torch.float32)
else:
    data_tensor = torch.tensor(data_tensor, dtype=torch.float32)


print(data_tensor.shape)


In [ ]:
import pandas as pd
import numpy as np
import itertools

def create_pairwise_binary_mask(matrix_cells_x_genes, gene_list):
    """
    Creates a dictionary of binary masks for each cell and pair of genes,
    including both forward, reverse, and self-pairs (which are all zeros).

    Args:
        matrix_cells_x_genes (np.ndarray): A 2D numpy array where rows are cells
                                          and columns are genes.
        gene_list (list): A list of strings containing the names of the genes,
                          in the same order as the columns in the matrix.

    Returns:
        dict: A dictionary where keys are gene pair strings (e.g., 'GeneA_GeneB')
              and values are 1D numpy arrays representing the binary mask for that pair
              across all cells.
    """
    binary_matrix = (matrix_cells_x_genes > 0).astype(int)
    num_cells, num_genes = binary_matrix.shape

    if len(gene_list) != num_genes:
        raise ValueError("The length of the gene_list must match the number of genes (columns) in the matrix.")

    pairwise_mask_dict = {}
    zero_vector = np.zeros(num_cells, dtype=int)
    for g_idx, gene_name in enumerate(gene_list):
        key = f"{gene_name}_{gene_name}"
        pairwise_mask_dict[key] = zero_vector

    gene_pairs_indices = list(itertools.combinations(range(num_genes), 2))
    for g1_idx, g2_idx in gene_pairs_indices:
        mask = binary_matrix[:, g1_idx] * binary_matrix[:, g2_idx]
        key_fwd = f"{gene_list[g1_idx]}_{gene_list[g2_idx]}"
        pairwise_mask_dict[key_fwd] = mask
        key_rev = f"{gene_list[g2_idx]}_{gene_list[g1_idx]}"
        pairwise_mask_dict[key_rev] = mask

    return pairwise_mask_dict

def dict_to_dataframe(mask_dict, column_order_list):
    """
    Converts a dictionary of binary masks into a pandas DataFrame,
    respecting a specified column order.

    Args:
        mask_dict (dict): A dictionary where keys are gene pair strings and
                          values are 1D numpy arrays (the masks).
        column_order_list (list): A list of gene pair strings specifying the
                                  desired order of the DataFrame columns.

    Returns:
        pd.DataFrame: A DataFrame with masks as columns, in the specified order.
    """
    # 1. Create a dictionary with only the ordered columns
    ordered_data = {col: mask_dict[col] for col in column_order_list if col in mask_dict}
    
    # 2. Check if all specified columns were found
    if len(ordered_data) != len(column_order_list):
        missing_columns = set(column_order_list) - set(ordered_data.keys())
        print(f"Warning: The following columns were not found in the mask dictionary: {missing_columns}")

    # 3. Create the DataFrame from the ordered dictionary
    df = pd.DataFrame(ordered_data)
    
    return df


counts = pd.DataFrame(adata.X, columns= adata.var_names)



In [ ]:
import numpy as np

def flatten_upper_triangular_excluding_diagonal(matrix):
    """
    Flattens the upper triangular part of a matrix, excluding the diagonal.

    Args:
        matrix: A list of lists or a NumPy array representing the matrix.

    Returns:
        A NumPy array containing the flattened upper triangular elements.
    """
    # Convert the input to a NumPy array for efficient operations.
    np_matrix = np.array(matrix)

    # Get the upper triangular part of the matrix, excluding the diagonal.
    # The 'k=1' argument specifies that the diagonal should not be included.
    upper_triangle = np.triu(np_matrix, k=1)

    # Flatten the resulting matrix.
    flattened_matrix = upper_triangle.flatten()

    # Filter out the zero values that were not part of the original matrix.
    # We use a boolean mask to keep only non-zero elements.
    result = flattened_matrix[flattened_matrix != 0]

    return result



In [ ]:
from scipy.cluster import hierarchy
from scipy.stats import spearmanr
from statsmodels.formula.api import quantreg

def get_hierarchical_clustering(adata):

    corr_matrix, _ = spearmanr(adata.X, axis=0)

    corr_matrix = np.corrcoef(adata.X.T)

    corr_dist = 1 - corr_matrix
    dist_linkage = hierarchy.average(corr_dist)

    df = pd.DataFrame({'cophenet':hierarchy.cophenet(dist_linkage), 'corr':  flatten_upper_triangular_excluding_diagonal(corr_matrix)})

    low_quantile_model = quantreg('corr ~ cophenet', df).fit(q=0.1)

    # np.sort is used to ensure the line is drawn smoothly from left to right
    x_sorted = np.sort(df['cophenet'])
    y_predicted = low_quantile_model.predict({'cophenet': x_sorted})

    return dist_linkage


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt



def fit_regression_clusterings(corr_matrix, dist_linkage, threshold=2):

    df = pd.DataFrame({'cophenet':hierarchy.cophenet(dist_linkage), 'corr':  flatten_upper_triangular_excluding_diagonal(corr_matrix)})
    df =df[df.cophenet<threshold]
    low_quantile_model = quantreg('corr ~ cophenet', df).fit(q=0.1)

    # np.sort is used to ensure the line is drawn smoothly from left to right
    df = df.sort_values('cophenet')
    x_sorted = df['cophenet']

    # model line
    y_predicted = low_quantile_model.predict({'cophenet': x_sorted})
    df['linear_model'] = y_predicted
    return df

def plot_regression(df):
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.scatter(df['cophenet'], df['corr'], alpha=0.7, label='Data Points')

    ax.plot(df['cophenet'], df['linear_model'], color='red', linewidth=2, label='10th Percentile Quantile Regression Line')
    ax.axhline(y=0.6, color='r', linestyle='--', label='y = 0.6')
    # Add labels and a legend for clarity
    plt.title('Quantile Regression with correlation threshold')
    plt.xlabel('Cophenet')
    plt.ylabel('Correlation')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:


def cut_clustering_and_gene_mapping(corr_matrix, dist_linkage, threshold, variable_names):
    clusters = hierarchy.fcluster(dist_linkage, t=threshold, criterion='distance')

    cluster_to_genes = {}
    for gene_name, cluster_id in zip(variable_names, clusters):
        if cluster_id not in cluster_to_genes:
            cluster_to_genes[cluster_id] = []
        cluster_to_genes[cluster_id].append(gene_name)

    high_average_similarity = {}
    high_average_similarity_idx = {}

    for cluster_id, gene_list in cluster_to_genes.items():
        if len(gene_list) > 1:
            # Get the sub-matrix of the correlation matrix for the genes in the cluster
            gene_indices = [variable_names.get_loc(g) for g in gene_list]
            cluster_corr_matrix = corr_matrix[np.ix_(gene_indices, gene_indices)]

            # Calculate the average of the upper triangle (excluding the diagonal)
            upper_triangle_indices = np.triu_indices_from(cluster_corr_matrix, k=1)
            average_similarity = np.mean(cluster_corr_matrix[upper_triangle_indices])

            #if average_similarity >= req_sim:
            print(f"Cluster {cluster_id}: {gene_list}")
            print(f"  Average Similarity: {average_similarity:.4f}")
            print("-" * 45)
            high_average_similarity[cluster_id] = gene_list
            high_average_similarity_idx[cluster_id] = gene_indices
    return high_average_similarity_idx, high_average_similarity


In [ ]:
from collections import OrderedDict
import random

def get_max_identity_class_size(high_average_similarity_idx):
    # order not important
    identity_class_sizes_l = []
    identity_class_sizes = {}
    for k in high_average_similarity_idx.keys():
        identity_class_sizes[k] = len(high_average_similarity_idx[k])
        identity_class_sizes_l.append(len(high_average_similarity_idx[k]))
    return np.max(identity_class_sizes_l)




def get_id_class_representative(data_dict: OrderedDict, keys):

    elements = []
    for key in keys:
        value = data_dict[key]
        
        # Check if the 'unused' list is empty
        if not value['unused']:
            value['unused'].extend(value['used'])
            value['used'].clear()
            random.shuffle(value['unused'])

        random_index = random.randint(0, len(value['unused']) - 1)
        element = value['unused'].pop(random_index)
        value['used'].append(element)
        elements.append(element)

    return elements, data_dict

In [ ]:
def create_dataset_assembly_dict(high_average_similarity_idx):
    id_class_dictionary = {}
    all_genes_in_identity_class = []
    for k in  high_average_similarity_idx:
        id_class_dictionary[k] = {'unused': high_average_similarity_idx[k].copy(), 'used' : []}
        all_genes_in_identity_class = all_genes_in_identity_class+high_average_similarity_idx[k]
    return id_class_dictionary, all_genes_in_identity_class

In [ ]:
from collections import OrderedDict
import random

def get_id_class_representative(data_dict: OrderedDict, keys) -> OrderedDict:

    elements = []
    for key in keys:
        value = data_dict[key]
        
        # Check if the 'unused' list is empty
        if not value['unused']:
            value['unused'].extend(value['used'])
            value['used'].clear()
            random.shuffle(value['unused'])

        random_index = random.randint(0, len(value['unused']) - 1)
        element = value['unused'].pop(random_index)
        value['used'].append(element)
        elements.append(element)

    return elements, data_dict


In [ ]:
corr_matrix = np.corrcoef(adata.X.T)

corr_dist = 1 - corr_matrix
dist_linkage = hierarchy.average(corr_dist)
df = fit_regression_clusterings(corr_matrix, dist_linkage, threshold=2)
plot_regression(df)
cluster_mapping, cluster_mapping_genes = cut_clustering_and_gene_mapping(corr_matrix, dist_linkage, 0.75, adata.var_names)
max_id_classes = get_max_identity_class_size(cluster_mapping)


id_class_dictionary, all_genes_in_id_class = create_dataset_assembly_dict(cluster_mapping)
other_genes = set(range(len(adata.var_names))) - set(all_genes_in_id_class)
keys = list(cluster_mapping.keys())

In [ ]:
cluster_mapping


In [ ]:

def create_model_zoo(data_tensor, id_class_dictionary, keys, other_genes, n_models = 4, n_epochs = 500):
    model_zoo = []
    variables_selected = []
    representatives = []
    for _ in range(n_models):

        elements, id_class_dictionary = get_id_class_representative(id_class_dictionary, keys)

        current_data_selection = list(other_genes)+list(elements)
        current_data = data_tensor[:,current_data_selection]
        data_train2, data_test2 = train_test_split(current_data,test_size=0.01, shuffle=True)

        trained_model2 = ZINBAutoencoder(input_dim=current_data.shape[1], latent_dim=10, dropout_rate = 0.02)
        trained_model2 = trained_model2.cuda()

        optimizer2 = torch.optim.Adam(trained_model2.parameters(), lr=1e-4)

        trained_model2 = train_autoencoder(
                trained_model2,
                data_train2.cuda(),
                optimizer2,
                num_epochs=n_epochs

            )
        model_zoo.append(trained_model2)
        variables_selected.append(current_data_selection)
        representatives.append(elements)
    return model_zoo, variables_selected, representatives




In [336]:
model_zoo, variables_selected, representatives = create_model_zoo(data_tensor, id_class_dictionary, keys, other_genes, n_models=50, n_epochs=500)

Epoch 1/500, Loss: 37.0872
Epoch 11/500, Loss: 5.0993
Epoch 21/500, Loss: 4.2926
Epoch 31/500, Loss: 3.9876
Epoch 41/500, Loss: 3.8438
Epoch 51/500, Loss: 3.7589
Epoch 61/500, Loss: 3.6955
Epoch 71/500, Loss: 3.6509
Epoch 81/500, Loss: 3.6097
Epoch 91/500, Loss: 3.5774
Epoch 101/500, Loss: 3.5484
Epoch 111/500, Loss: 3.5207
Epoch 121/500, Loss: 3.4984
Epoch 131/500, Loss: 3.4772
Epoch 141/500, Loss: 3.4640
Epoch 151/500, Loss: 3.4501
Epoch 161/500, Loss: 3.4365
Epoch 171/500, Loss: 3.4241
Epoch 181/500, Loss: 3.4091
Epoch 191/500, Loss: 3.4000
Epoch 201/500, Loss: 3.3915
Epoch 211/500, Loss: 3.3833
Epoch 221/500, Loss: 3.3737
Epoch 231/500, Loss: 3.3669
Epoch 241/500, Loss: 3.3593
Epoch 251/500, Loss: 3.3565
Epoch 261/500, Loss: 3.3490
Epoch 271/500, Loss: 3.3421
Epoch 281/500, Loss: 3.3380
Epoch 291/500, Loss: 3.3316
Epoch 301/500, Loss: 3.3257
Epoch 311/500, Loss: 3.3204
Epoch 321/500, Loss: 3.3129
Epoch 331/500, Loss: 3.3090
Epoch 341/500, Loss: 3.3052
Epoch 351/500, Loss: 3.3010
Ep

In [337]:
gene_to_position_mapper = {}
max_un = (len(variables_selected[0])-len(representatives[0]))
for i in range(len(gene_names)):
    if i < (len(variables_selected[0])-len(representatives[0])):
        gene_to_position_mapper[i] = i

start_idx = max_un
current_idx = max_un
for k in cluster_mapping:
    for elem in cluster_mapping[k]:
        gene_to_position_mapper[current_idx] =start_idx 
        current_idx = current_idx+1
    start_idx = start_idx+1


In [338]:
def set_mu_true(model_zoo):
    # forward only mu
    for mo in model_zoo:
        mo.forward_mu_only = True
        mo.forward_theta_only = False
        mo.latent_only = False
        mo.forward_pi_only = False
    return model_zoo

def all_false(model_zoo):
    # forward only mu
    for mo in model_zoo:
        mo.forward_mu_only = False
        mo.forward_theta_only = False
        mo.latent_only = False
        mo.forward_pi_only = False
    return model_zoo

model_zoo  = all_false(model_zoo)


config.raw_attribution = True
config.percentile = 55
grn_adata = wrapper(model_zoo, data_tensor.cuda(), gene_names, variables_selected,  config)



100%|██████████| 461/461 [10:25<00:00,  1.36s/it]


AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 1000 × 461
AnnData object with n_obs × n_vars = 100

In [339]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
grn_adata.obs['grn'] = pd.Categorical(adata.obs['grn'])
grn_adata[:, pd.DataFrame(grn_adata[grn_adata.obs['grn']==2.0].layers['counter']).sum()>200].var.merge(net1)

,source,target,edge
0,AR,AATF,AR_AATF
1,AR,ABCC4,AR_ABCC4
2,AR,ABCE1,AR_ABCE1
3,AR,ADAM9,AR_ADAM9
4,AR,AGR3,AR_AGR3
...,...,...,...
118,AR,UGT2B17,AR_UGT2B17
119,AR,URI1,AR_URI1
120,AR,USP26,AR_USP26
121,AR,VAPA,AR_VAPA


In [341]:
pd.DataFrame(grn_adata[grn_adata.obs['grn']==2.0].layers['counter']).sum()

0        487.840
1         27.540
2          0.846
3         11.200
4         79.020
           ...  
212516    50.840
212517    42.460
212518   485.880
212519   394.380
212520   133.540
Length: 212521, dtype: float64

In [357]:
net0.groupby('source').count()

,target,edge
source,,
ESRRA,15,15
ESRRG,9,9
FOXD4,1,1
KDM3B,1,1
KLF16,1,1
MED23,1,1
NCOR2,13,13
NR0B1,9,9
NR2F1,21,21


In [364]:
grn_adata[:, pd.DataFrame(grn_adata.layers['counter']).sum()>250].var

,source,target
index,,
AATF_AATF,AATF,AATF
ABCC4_AATF,ABCC4,AATF
ABCE1_AATF,ABCE1,AATF
ABCG1_AATF,ABCG1,AATF
ABCG8_AATF,ABCG8,AATF
...,...,...
ZBTB20_ZNF746,ZBTB20,ZNF746
ZBTB22_ZNF746,ZBTB22,ZNF746
ZBTB7C_ZNF746,ZBTB7C,ZNF746


In [299]:
def attribution_one_target( 
        target_gene,
        lrp_model,
        input_data,
        background,
        selected_variables,
        xai_type='lrp-like',
        randomize_background = False):
    
    attributions_list = []
    for m in range(len(lrp_model)):

        # Randomize backgorund for each round
        if randomize_background:
            current_background = background[:, selected_variables[m]]
            current_background = shuffle_each_column_independently(current_background)

        current_data = input_data[:, selected_variables[m]]
        model = lrp_model[m]
        #for _ in range(num_iterations):
        if xai_type == 'lrp-like':
            attribution = model.attribute(current_data, target=target_gene)
                
        elif xai_type == 'shap-like':
            attribution = model.attribute(current_data, baselines = current_background, target = target_gene)

        attributions_list.append(attribution.detach().cpu().numpy())
    return attributions_list


def attribution_one_target_one_model( 
        target_gene,
        lrp_model,
        input_data,
        background,
        selected_variables,
        xai_type='lrp-like',
        randomize_background = False):
    
    # Randomize backgorund for each round
    if randomize_background:
        background = shuffle_each_column_independently(background)

    model = lrp_model
    #for _ in range(num_iterations):
    if xai_type == 'lrp-like':
        #print(input_data)
        #print(target_gene)
        attribution = model.attribute(input_data, target=target_gene)
            
    elif xai_type == 'shap-like':
        attribution = model.attribute(input_data, baselines = background, target = target_gene)

    return attribution.detach().cpu().numpy()

def get_top_edges_per_cell(grn_adata, top_edges):
    
    counters = np.zeros((grn_adata.shape))

    idex = grn_adata.shape[1]-top_edges
    b = np.argpartition(grn_adata, idex, axis=1)[:, idex:]

    np.put_along_axis(counters, b, 1, axis=1)
    return counters

    
def assemble_attributions(current_attr, variables_selected, a_shape):
    assembled_attributions = np.zeros(a_shape)
    column_counter = np.zeros(a_shape[1])
    for i in range(len(variables_selected)):
        for j in range(len(variables_selected[i])):
            assembled_attributions[:, variables_selected[i][j]] += current_attr[i][:, j]
            column_counter[variables_selected[i][j]] +=1
    for i in range(len(column_counter)):
        if column_counter[i] == 0:
            column_counter[i] =1
    assembled_attributions = assembled_attributions/column_counter
    return assembled_attributions, column_counter

def wrapper(models, data_train_full_tensor, gene_names, variables_selected, config):

    data = data_train_full_tensor.detach().cpu().numpy()
    tms = []
    name_list = []
    target_names = []


    for trained_model in models:        
        trained_model.forward_mu_only = True
        explainer, xai_type = get_explainer(trained_model, config.xai_method, config.raw_attribution)
        tms.append(explainer)

    attributions = []
    all_counter = []
    ## ATTRIBUTIONS
    for g in tqdm(range(data_train_full_tensor.shape[1])):
    #for g in tqdm(range(len(variables_selected[0]))):
        current_gene = gene_to_position_mapper[g]
        current_attributions = []
        counter_list = []
        for m in range(len(tms)):

            attributions_list = attribution_one_target_one_model(
                current_gene,
                tms[m], # Select the correct model
                data_train_full_tensor[:, variables_selected[m]],
                data_train_full_tensor[:, variables_selected[m]],
                variables_selected, 
                xai_type=xai_type,
                randomize_background = True)
            counters = get_top_edges_per_cell(attributions_list, 100)
            counter_list.append(counters)
            current_attributions.append(attributions_list)

        current_attributions, column_counter = assemble_attributions(current_attributions, variables_selected, data_train_full_tensor.shape)
        counter_list, col = assemble_attributions(counter_list, variables_selected, data_train_full_tensor.shape)
        attributions.append(current_attributions)
        all_counter.append(counter_list)



    ## AGGREGATION: REPLACE LIST BY AGGREGATED DATA
    # for i in range(len(attributions)):
    #     # CURRENTLY MEAN
    #     attributions[i] = aggregate_attributions(attributions[i], strategy=config.aggregation_strategy )
    

    ## PENALIZE:
    if config.penalty != 'None':
        penalty_matrix = compute_correlation_metric(data, cor_type=config.penalty)
        for i in range(len(attributions)):
            # CURRENTLY MEAN
            attributions[i] = np.dot(attributions[i], (1-penalty_matrix))

    
    ## CLUSTERING: CLUSTER EACH TARGET INDVIDUALLY
    for i in range(len(attributions)):
 
        attributions[i] = ad.AnnData(attributions[i])
        print(attributions[i])
        sc.pp.scale(attributions[i])
        try:
            sc.pp.pca(attributions[i],n_comps=50)
        except:
            try:
                sc.pp.pca(attributions[i],n_comps=50 )
            except:
                continue
            
        sc.pp.neighbors(attributions[i], n_neighbors=15)
        sc.tl.leiden(attributions[i], resolution=0.1)

        #clusterings[f'T_{gene_names[i]}'] = np.array(attributions[i].obs['leiden'])

    #EDGE SELECTION:
    for i in range(len(attributions)):
        edge_indices = get_edges(attributions[i], use_differential=config.use_differential, percentile=config.percentile)
        name_list = name_list + list(gene_names[edge_indices])
        target_names = target_names+[gene_names[i]]* len(edge_indices)
        attributions[i] = attributions[i][:,edge_indices].X
        all_counter[i] = all_counter[i][:, edge_indices]

    attributions = np.hstack(attributions)
    all_counter = np.hstack(all_counter)

    
    index_list = [f"{s}_{t}" for (s, t) in zip(name_list, target_names)]
    cou = pd.DataFrame({'index': index_list, 'source':name_list, 'target':target_names})
    cou = cou.set_index('index')

    #clusterings = pd.DataFrame(clusterings)

    #grn_adata = attribution_to_anndata(attributions, var=cou, obs = clusterings)
    grn_adata = attribution_to_anndata(attributions, var=cou)
    grn_adata.layers['counter'] = all_counter
    return grn_adata





In [292]:
counter = np.zeros((3,6))
ar = np.array([[1,2,3,4,5,6], [2,2,5,1,2,3], [1,212,2,2,1,6]])
p = np.argpartition(np.array([[1,2,3,4,5,6], [2,2,5,1,2,3], [1,212,2,2,1,6]]), 4, axis=1)[:, 4:]
print(p)
np.put_along_axis(counter, p, 1, axis=1)
counter

[[4 5]
 [5 2]
 [5 1]]


array([[0., 0., 0., 0., 1., 1.],
       [0., 0., 1., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1.]])

In [ ]:
import scipy.stats as st
import itertools

def find_consistent_pairs(grn_adata, gene_names):
    """
    Creates a dictionary of binary masks for each cell and pair of genes,
    including both forward, reverse, and self-pairs (which are all zeros).

    Args:
        matrix_cells_x_genes (np.ndarray): A 2D numpy array where rows are cells
                                          and columns are genes.
        gene_list (list): A list of strings containing the names of the genes,
                          in the same order as the columns in the matrix.

    Returns:
        dict: A dictionary where keys are gene pair strings (e.g., 'GeneA_GeneB')
              and values are 1D numpy arrays representing the binary mask for that pair
              across all cells.
    """

    num_cells, num_edges = grn_adata.X.shape

    pairwise_mask_dict = {}

    gene_pairs_indices = list(itertools.combinations(gene_names, 2))
    for g1_idx, g2_idx in gene_pairs_indices:
        pairwise_mask_dict[f"{g1_idx}_{g2_idx}"] = st.spearmanr(grn_adata[:, [f"{g1_idx}_{g2_idx}", f"{g2_idx}_{g1_idx}"]].X).statistic
        # add reverse bc I am lazy
        pairwise_mask_dict[f"{g2_idx}_{g1_idx}"] = pairwise_mask_dict[f"{g1_idx}_{g2_idx}"]
    return pairwise_mask_dict

In [ ]:

pariwise_consistency_1 = find_consistent_pairs(grn_adata[grn_adata.obs['grn'] == 1.0, :].copy(), gene_names1)
pariwise_consistency_2 = find_consistent_pairs(grn_adata[grn_adata.obs['grn'] == 2.0, :].copy(), gene_names1)

In [ ]:
pairwise_consistency_1 = pd.DataFrame(pariwise_consistency_1.items())
pairwise_consistency_2 = pd.DataFrame(pariwise_consistency_2.items())

In [ ]:
pairwise_consistency_1.columns = ['edge', 'spearman']
pairwise_consistency_2.columns = ['edge', 'spearman']



In [ ]:
consisten_edges2 = grn_adata[:, pairwise_consistency_2[pairwise_consistency_2.spearman>0.2].edge].var
grn_adata[:, pairwise_consistency_1[pairwise_consistency_1.spearman>0.3].edge].var
grn_adata[:, pairwise_consistency_2[pairwise_consistency_2.spearman>0.2].edge].var

In [ ]:
grn_adata.obs['grn'] = pd.Categorical(adata.obs['grn'])

In [ ]:
def get_top_edges_per_cell(grn_adata, top_edges):
    
    idex = grn_adata.shape[1]-top_edges
    b = np.argpartition(grn_adata.X, idex, axis=1)    # top 3 values from each row
    top_idx = b[:,idex:]


    scgenerai_var_index_np = np.array(grn_adata.var)
    top_edges_per_cell = pd.DataFrame(np.concat(scgenerai_var_index_np[top_idx]))
    top_edges_per_cell.columns = ['source', 'target', 'n_cells']
    top_edges_per_cell['cell_barcode'] = np.repeat(grn_adata.obs.index, repeats=top_edges)
    return top_edges_per_cell





In [ ]:
#grn_adata = downstream_recipe(grn_adata)
#grn_adata.var.columns = ['source', 'target', 'n_cells']
# Example UsageCreate the binary mask with both forward and reverse pairs and named strings
mask_full = create_pairwise_binary_mask(counts.values, counts.columns)
mask_full = dict_to_dataframe(mask_full, grn_adata.var_names)

In [ ]:

grn_adata.layers['mask'] = mask_full
grn_adata.layers['masked'] = np.multiply(grn_adata.X, grn_adata.layers['mask'])


In [ ]:
grn_adata.obs['grn'] = pd.Categorical(adata.obs['grn'])
sc.tl.rank_genes_groups(grn_adata, "grn", method="t-test", use_raw=False, rankby_abs=True, layer='masked')
sc.tl.filter_rank_genes_groups(grn_adata, min_fold_change=3)
de_grn = sc.get.rank_genes_groups_df(grn_adata, group='2.0')
de_grn[['source', 'target']] = de_grn['names'].str.split('_', expand=True)

In [ ]:
de_grn[de_grn.names == 'AR_EPB41L4B']

In [ ]:
de_grn[de_grn.names == 'EPB41L4B_AR']

de_grn

In [ ]:
de_grn.merge(net1)

In [ ]:
sc.pl.violin(adata, keys=[ 'PCK1', 'TRNT1', 'AR', 'EPB41L4B'], groupby='grn')

In [ ]:
sc.pl.violin(grn_adata, keys=['PCK1_TRNT1', 'AR_SORD', 'AR_EPB41L4B'], use_raw=False, groupby='grn')

In [ ]:
net0 =  nets[0]
net0['edge'] = net0['source']+'_'+net0['target']

net1 =  nets[1]
net1['edge'] = net1['source']+'_'+net1['target']



In [ ]:
adata.obs['grn'] = pd.Categorical(adata.obs['grn'])
sc.pp.log1p(adata)
sc.tl.rank_genes_groups(adata, "grn", method="wilcoxon", use_raw=False, rankby_abs=True)
sc.tl.filter_rank_genes_groups(adata, min_fold_change=3)
de = sc.get.rank_genes_groups_df(adata, group='2.0')

In [ ]:
grn_adata[0:50000].merge(net1)

In [ ]:
de_grn[0:50000]

In [ ]:
adata = downstream_recipe(adata)
sc.pl.umap(adata, color = ['grn', 'RASGRP3','BDP1', 'CREBBP','VSX2', 'TBP','TAF5', 'FLI1', 'KAT2B', 'IGFBP7', 'AEBP1'])

In [ ]:
grn_adata.obs['AEBP1'] = adata[:, 'AEBP1'].X.copy().flatten()
grn_adata.obs['IGFBP7'] = adata[:, 'IGFBP7'].X.copy().flatten()

grn_adata.obs['BBX'] = adata[:, 'BBX'].X.copy().flatten()
grn_adata.obs['PHF2'] = adata[:, 'PHF2'].X.copy().flatten()
grn_adata.obs['TAF10'] = adata[:, 'TAF10'].X.copy().flatten()
sc.pl.umap(grn_adata, color= ['IGFBP7_AEBP1', 'leiden', 'AEBP1', 'IGFBP7', 'IGFBP7_BBX', 'PHF2_TAF10', 'TAF10', 'PHF2'], use_raw=False)

In [ ]:
de[(de.pvals_adj<0.01)].merge(net0, left_on='names', right_on='edge')

In [ ]:
de[de.names == 'KAT2B']

In [ ]:
sc.pl.violin(grn_adata, keys= ['TAT_CNOT4', 'ZBTB8B_ZW10', 'GATA3_BLCAP', 'TAT_ST8SIA2', 'NF1_ADD1', 'GATA3_FNDC3B'], use_raw=False, groupby='grn')

In [ ]:
sc.pl.violin(adata, keys=['TAT','ST8SIA2', 'ZBTB8B', 'ZW10', 'GATA3', 'BLCAP', 'CNOT4', 'ADD1', 'NF1', 'FNDC3B'], groupby='grn')

In [ ]:
gene = 'CNOT4'
print(net0[net0.source == gene])
print(net0[net0.target == gene])
print(net1[net1.source == gene])
print(net1[net1.target == gene])


In [ ]:
net0[net0.target == 'NF1']

In [ ]:
grn_adata.obs['grn'] = pd.Categorical(adata.obs['grn'])
top = get_top_edges_per_cell(grn_adata[grn_adata.obs['grn'] == 2.0], 1000)
top['edgenames'] = top['source']+'_'+top['target']

In [ ]:

top = top.loc[:, ['source', 'target']].groupby(['source', 'target']).value_counts().reset_index()


In [ ]:
top.columns = ['source', 'target', 'counter']
top.merge(nets[1]).hist('counter')

In [ ]:
top.hist('counter')

In [ ]:
top.sort_values('counter', ascending=False)[0:10000].loc[:, ['source']].groupby('source').value_counts().reset_index()

In [ ]:
grn_adata.obs['grn'] = pd.Categorical(adata.obs['grn'])
adata.obs['grn'] = pd.Categorical(adata.obs['grn'])
sc.pl.violin(adata, keys=['ZBTB8A', 'ZBTB8B', 'CDKN1A'], groupby='grn')

In [ ]:
mapping = {}
for key in cluster_mapping:
    for i in cluster_mapping[key]:
        gene = gene_names[i]
        indx = [g for g in cluster_mapping[key] if gene_names[g]!=gene]
        mapping[gene] = gene_names[indx]

In [ ]:
ext.loc[:, ['source', 'target']].duplicated().sum()


In [ ]:
def extend_edges(mapping, top):
    expanded = []
    for i, edges in top.iterrows():
        # Add combinations all all mapped edges in the lists
        if edges['source'] in mapping and edges['target'] in mapping:
            for se in mapping[edges['source']]:
                for st in mapping[edges['target']]:
                    expanded.append([se, st])
        if edges['source'] in mapping:
            for se in mapping[edges['source']]:
                expanded.append([se, edges['target']])
        if edges['target'] in mapping:
            for se in mapping[edges['target']]:
                expanded.append([edges['source'], se])

    expanded = pd.DataFrame(expanded)
    expanded.columns = ['source', 'target']
    all_edges = pd.concat([top, expanded])
    return all_edges


In [ ]:
grn_adata = downstream_recipe(grn_adata)



In [ ]:
sub = grn_adata[adata.obs.grn == 2.0, grn_adata.var.source == 'MTA1']
gatatop = get_top_edges_per_cell(sub, 75)
gatatop = gatatop.loc[:, ['source', 'target']].groupby(['source', 'target']).value_counts().reset_index().sort_values('count', ascending=False)
gatatop = gatatop[gatatop['count']>250]
gatatop = extend_edges(mapping, gatatop)
gatatop

In [ ]:
gatatop.merge(nets[0])

In [ ]:
nets[0][nets[0].source == 'MTA1']

In [ ]:
nets[0][nets[0].source == 'GATA3']

In [ ]:
mapping

In [ ]:
np.unique(all_edges.sort_values('count', ascending=False)[0:200].source)

In [ ]:
all_edges.sort_values('count', ascending=False)[0:10000].hist('count')

In [ ]:
nets0 = nets[0]
nets0[nets0.target == 'WTAP']

In [ ]:
all_edges.sort_values('count', ascending=False).hist('count')

In [ ]:
adata.obs.grn = pd.Categorical(adata.obs.grn)

In [ ]:
all_edges[all_edges['count']>20]

In [ ]:
all_edges[all_edges['count']>20].sort_values('count', ascending=False).source.value_counts()

In [ ]:
sc.pl.violin(adata, keys = ['TMPRSS5', 'ITGB3BP'], groupby= 'grn')

In [ ]:
sc.pl.violin(adata, keys = np.unique(all_edges.sort_values('count', ascending=False)[0:200].source)[0:5], groupby= 'grn')

In [ ]:
sc.pl.violin(adata, keys = np.unique(all_edges.sort_values('count', ascending=False)[0:200].source)[5:10], groupby= 'grn')

In [ ]:
sc.pl.violin(adata, keys = np.unique(all_edges.sort_values('count', ascending=False)[0:200].source)[10:15], groupby= 'grn')

In [ ]:
sc.pl.violin(adata, keys = np.unique(all_edges.sort_values('count', ascending=False)[0:200].source)[15:20], groupby= 'grn')

In [ ]:
import pandas as pd
import numpy as np

def binarize_and_compute_contingency(df_counts):
    """
    Binarizes raw gene count data and computes pairwise contingency tables efficiently.

    Args:
        df_counts (pd.DataFrame): A DataFrame with genes as rows and samples as columns.

    Returns:
        dict: A dictionary containing four DataFrames for the contingency table values:
              'a' (both genes expressed), 'b' (gene A expressed, gene B not),
              'c' (gene A not, gene B expressed), and 'd' (neither expressed).
    """

    # Step 1: Binarize the gene expression data
    # Convert counts to 1 (expressed) or 0 (not expressed)
    df_binary = (df_counts > 0).astype(int)

    # Step 2: Compute pairwise contingency using matrix multiplication
    # 'a': Both genes are expressed (count > 0 in the same sample)
    a_matrix = df_binary.dot(df_binary.T)

    # 'd': Neither gene is expressed (count == 0 in the same sample)
    # We invert the binary matrix to find where both are 0
    df_binary_inverted = 1 - df_binary
    d_matrix = df_binary_inverted.dot(df_binary_inverted.T)

    # Calculate marginal sums for 'b' and 'c'
    # Row-wise sum gives total number of samples where each gene is expressed
    expressed_counts = df_binary.sum(axis=1)

    # 'b': Gene A is expressed, Gene B is not
    # This is calculated as (total samples where Gene A is expressed) - (samples where both are expressed)
    b_matrix = expressed_counts.to_frame().values - a_matrix

    # 'c': Gene B is expressed, Gene A is not
    # This is calculated as (total samples where Gene B is expressed) - (samples where both are expressed)
    # We use a transpose here to align the dimensions correctly
    c_matrix = expressed_counts.to_frame().T.values - a_matrix

    # Correcting for floating point inaccuracies by rounding to nearest integer
    b_matrix = b_matrix.round().astype(int)
    c_matrix = c_matrix.round().astype(int)

    return {
        'a': pd.DataFrame(a_matrix, index=df_counts.index, columns=df_counts.index),
        'b': pd.DataFrame(b_matrix, index=df_counts.index, columns=df_counts.index),
        'c': pd.DataFrame(c_matrix, index=df_counts.index, columns=df_counts.index),
        'd': pd.DataFrame(d_matrix, index=df_counts.index, columns=df_counts.index)
    }






In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from multiprocessing import Pool

def compute_single_chi2(args):
    """
    Helper function to compute chi-squared for a single pair of genes.
    This function is designed to be used with multiprocessing.Pool.
    """
    contingency_tables, genea, geneb = args
    genenames = contingency_tables['a'].columns
    contingency_dict = {
        'expressed': {'expressed': contingency_tables['a'].loc[genenames[genea], genenames[geneb]],
                      'not_expressed': contingency_tables['b'].loc[genenames[genea], genenames[geneb]]},
        'not_expressed': {'expressed': contingency_tables['c'].loc[genenames[genea], genenames[geneb]],
                          'not_expressed': contingency_tables['d'].loc[genenames[genea], genenames[geneb]]}
    }
    contingency_df2 = pd.DataFrame(contingency_dict)
    try:
        cont = chi2_contingency(contingency_df2)
        return genea, geneb, cont.pvalue, cont.statistic
    except ValueError:
        return genea, geneb, np.nan, np.nan

def compute_stats_mat_parallel(contingency_tables):
    """
    Computes the chi-squared p-values and test statistics in parallel
    using the multiprocessing library.
    """
    loop_end = contingency_tables['a'].shape[1]
    pvalues = np.zeros((loop_end, loop_end))
    teststat = np.zeros((loop_end, loop_end))
    
    tasks = [(contingency_tables, genea, geneb) for genea in range(loop_end) for geneb in range(genea + 1, loop_end)]
    
    with Pool() as pool:
        results = pool.map(compute_single_chi2, tasks)
    
    for genea, geneb, pvalue, statistic in results:
        pvalues[genea, geneb] = pvalue
        teststat[genea, geneb] = statistic
        
    pvalues = pvalues + pvalues.T
    teststat = teststat + teststat.T


    np.fill_diagonal(pvalues, 1)
    np.fill_diagonal(teststat, 0)


    return pvalues, teststat, gene_names





In [ ]:
import pandas as pd
import numpy as np

def binarize_and_compute_contingency(df_counts):
    """
    Binarizes raw gene count data with a conditional logic and computes
    pairwise contingency tables efficiently.

    Args:
        df_counts (pd.DataFrame): A DataFrame with genes as rows and samples as columns.

    Returns:
        pd.DataFrame: A DataFrame with binarized gene expression data.
    """

    # Determine the number of samples (columns)
    num_samples = df_counts.shape[1]

    # Step 1: Binarize the gene expression data based on a condition
    df_binary = pd.DataFrame(index=df_counts.index, columns=df_counts.columns, dtype=int)
    for gene in df_counts.index:
        gene_data = df_counts.loc[gene]
        # Count the number of zeros
        zero_count = (gene_data == 0).sum()

        if zero_count / num_samples > 0.10:
            # Case 1: More than 10% zeros, binarize as count > 0
            df_binary.loc[gene] = (gene_data > 0).astype(int)
        else:
            # Case 2: 10% or fewer zeros, binarize using the median
            # The labels=False argument ensures pd.qcut returns integer bin codes (0, 1, 2, 3)
            # The duplicates='drop' handles cases where bin edges might be the same.
            df_binary.loc[gene] = pd.qcut(gene_data, q=4, labels=False, duplicates='drop')

    return df_binary

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

def compute_single_chi2(args):
    """
    Helper function to compute chi-squared for a single pair of genes
    by directly computing a crosstab on the binarized data.
    """
    contingency_table, genea_index, geneb_index = args

    # Chi-squared test and return p-value and statistic
    try:
        cont = chi2_contingency(contingency_table)
        return genea_index, geneb_index, cont.pvalue, cont.statistic
    except ValueError:
        return genea_index, geneb_index, np.nan, np.nan

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from multiprocessing import Pool

def compute_single_chi2(args):
    """
    Helper function to compute chi-squared for a single pair of genes
    from a pre-computed contingency table.
    """
    contingency_table, genea_index, geneb_index = args
    
    # Chi-squared test and return p-value and statistic
    try:
        cont = chi2_contingency(contingency_table)
        return genea_index, geneb_index, cont.pvalue, cont.statistic
    except ValueError:
        return genea_index, geneb_index, np.nan, np.nan

def compute_stats_mat_parallel(df_binary):
    """
    Computes the chi-squared p-values and test statistics in parallel
    by first computing all crosstabs before parallelization.
    """
    loop_end = df_binary.shape[0]
    gene_names = df_binary.index
    
    pvalues = np.zeros((loop_end, loop_end))
    teststat = np.zeros((loop_end, loop_end))

    print("compute oontinencies")
    tasks = []
    for genea in range(loop_end):
        for geneb in range(genea + 1, loop_end):
            gene_a_data = df_binary.iloc[genea]
            gene_b_data = df_binary.iloc[geneb]
            
            # Create the contingency table using pandas.crosstab
            contingency_table = pd.crosstab(gene_a_data, gene_b_data)
            
            # Add the pre-computed table and indices to the task list
            tasks.append((contingency_table, genea, geneb))

    print('Compute tests')
    with Pool() as pool:
        results = pool.map(compute_single_chi2, tasks)
    
    # Step 3: Populate the result matrices
    for genea, geneb, pvalue, statistic in results:
        pvalues[genea, geneb] = pvalue
        teststat[genea, geneb] = statistic
        
    # Symmetrize the matrices
    pvalues = pvalues + pvalues.T
    teststat = teststat + teststat.T

    np.fill_diagonal(pvalues, 1)
    np.fill_diagonal(teststat, 0)

    return pvalues, teststat, gene_names

In [ ]:
def reformat(pvalues, teststat):

    pvalues = pd.DataFrame(pvalues)
    teststat = pd.DataFrame(teststat)
    pvalues.columns = gene_names
    teststat.columns = gene_names
    pvalues.index = gene_names
    teststat.index = gene_names
    pvalues = pvalues.melt(ignore_index=False).reset_index()
    pvalues.columns = ['source', 'target', 'pvalue']
    teststat = teststat.melt(ignore_index=False).reset_index()
    teststat.columns = ['source', 'target', 'teststat']
    return pvalues, teststat

In [ ]:
import pandas as pd
import numpy as np
import itertools

def create_pairwise_binary_mask(matrix_cells_x_genes, gene_list):
    """
    Creates a dictionary of binary masks for each cell and pair of genes,
    including both forward, reverse, and self-pairs (which are all zeros).

    Args:
        matrix_cells_x_genes (np.ndarray): A 2D numpy array where rows are cells
                                          and columns are genes.
        gene_list (list): A list of strings containing the names of the genes,
                          in the same order as the columns in the matrix.

    Returns:
        dict: A dictionary where keys are gene pair strings (e.g., 'GeneA_GeneB')
              and values are 1D numpy arrays representing the binary mask for that pair
              across all cells.
    """
    binary_matrix = (matrix_cells_x_genes > 0).astype(int)
    num_cells, num_genes = binary_matrix.shape

    if len(gene_list) != num_genes:
        raise ValueError("The length of the gene_list must match the number of genes (columns) in the matrix.")

    pairwise_mask_dict = {}
    zero_vector = np.zeros(num_cells, dtype=int)
    for g_idx, gene_name in enumerate(gene_list):
        key = f"{gene_name}_{gene_name}"
        pairwise_mask_dict[key] = zero_vector

    gene_pairs_indices = list(itertools.combinations(range(num_genes), 2))
    for g1_idx, g2_idx in gene_pairs_indices:
        mask = binary_matrix[:, g1_idx] * binary_matrix[:, g2_idx]
        key_fwd = f"{gene_list[g1_idx]}_{gene_list[g2_idx]}"
        pairwise_mask_dict[key_fwd] = mask
        key_rev = f"{gene_list[g2_idx]}_{gene_list[g1_idx]}"
        pairwise_mask_dict[key_rev] = mask

    return pairwise_mask_dict

def dict_to_dataframe(mask_dict, column_order_list):
    """
    Converts a dictionary of binary masks into a pandas DataFrame,
    respecting a specified column order.

    Args:
        mask_dict (dict): A dictionary where keys are gene pair strings and
                          values are 1D numpy arrays (the masks).
        column_order_list (list): A list of gene pair strings specifying the
                                  desired order of the DataFrame columns.

    Returns:
        pd.DataFrame: A DataFrame with masks as columns, in the specified order.
    """
    # 1. Create a dictionary with only the ordered columns
    ordered_data = {col: mask_dict[col] for col in column_order_list if col in mask_dict}
    
    # 2. Check if all specified columns were found
    if len(ordered_data) != len(column_order_list):
        missing_columns = set(column_order_list) - set(ordered_data.keys())
        print(f"Warning: The following columns were not found in the mask dictionary: {missing_columns}")

    # 3. Create the DataFrame from the ordered dictionary
    df = pd.DataFrame(ordered_data)
    
    return df



# Example UsageCreate the binary mask with both forward and reverse pairs and named strings
mask_full = create_pairwise_binary_mask(counts.values, counts.columns)
mask_full = dict_to_dataframe(mask_full, grn_adata.var_names)


In [ ]:
mask_full.sum()

In [ ]:
net1[net1.edge.isin(mask_full.iloc[:, np.where(mask_full.sum()<100)[0]].columns)]

In [ ]:
mask_full.iloc[:, np.where(mask_full.sum()<100)[0]].columns

In [ ]:
grn_adata.obs['grn'] = pd.Categorical(adata.obs['grn'])

In [ ]:
config.raw_attribution = True

In [ ]:
nets[0].merge(grn_adata.var, left_on=['source', 'target'], right_on=['source', 'target'])

In [ ]:
pariwise_consistency_1 = find_consistent_pairs(grn_adata[grn_adata.obs['grn'] == 1.0, :].copy(), adata.var_names)
pariwise_consistency_2 = find_consistent_pairs(grn_adata[grn_adata.obs['grn'] == 2.0, :].copy(), adata.var_names)

In [ ]:
consistencies2 = pd.DataFrame(pariwise_consistency_2.items())
consistencies2.columns = ['key', 'value']

consistencies1 = pd.DataFrame(pariwise_consistency_1.items())
consistencies1.columns = ['key', 'value']



In [ ]:
threshold = 0.5
#consistent_edges= np.unique(list(consistencies1[consistencies1.value>threshold].key)+ list((consistencies1[consistencies2.value>threshold].key)))
consistent_edges= np.unique(list(consistencies1[np.abs(consistencies1.value)>threshold].key)+ list((consistencies1[np.abs(consistencies2.value)>threshold].key)))

print(len(consistent_edges))

grn_sub = grn_adata[:, mask_full.sum(axis = 0)>0]

grn_sub.var_names.isin(consistent_edges)
grn_sub = grn_sub[:, grn_sub.var_names.isin(consistent_edges)]
print(grn_sub)

nets[0].merge(grn_sub.var, left_on=['source', 'target'], right_on=['source', 'target'])

In [ ]:
from src.evaluation.compute_metrics import  get_top_edges_per_cell, get_top_edges_per_cell_per_cluster, downstream_recipe

In [ ]:
mask_full

grn_adata.layers['mask'] = mask_full.values

In [ ]:
non_zero1 = grn_adata.layers['mask'][adata.obs['grn']==1.0].sum(axis=0)
non_zero2 = grn_adata.layers['mask'][adata.obs['grn']==2.0].sum(axis=0)

In [ ]:
consistencies1

In [ ]:
print(nets[0].shape)
print(nets[1].shape)

In [ ]:
import scipy.stats as st
pd.set_option('display.float_format', lambda x: '%.3f' % x)



In [ ]:
grn_sub

In [ ]:
grn_adata.obs['grn'] = pd.Categorical(adata.obs['grn'])

In [ ]:
grn_adata[grn_adata.obs['T_ACADVL'] == '1', ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3']]

In [ ]:
df = pd.DataFrame(st.spearmanr(grn_adata[:, ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3',  'BATF2_CCNA2', 'CCNA2_BATF2','TAF1_CCNA2', 'CCNA2_TAF1','POLI_CCNA2', 'CCNA2_POLI' ]].X).statistic)
df.columns = ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3', 'BATF2_CCNA2', 'CCNA2_BATF2','TAF1_CCNA2', 'CCNA2_TAF1', 'POLI_CCNA2', 'CCNA2_POLI' ]
df.index = ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3', 'BATF2_CCNA2', 'CCNA2_BATF2','TAF1_CCNA2', 'CCNA2_TAF1', 'POLI_CCNA2', 'CCNA2_POLI' ]
df

In [ ]:
df = pd.DataFrame(st.spearmanr(grn_adata[grn_adata.obs['grn'] == 1.0, ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3',  'BATF2_CCNA2', 'CCNA2_BATF2','TAF1_CCNA2', 'CCNA2_TAF1','POLI_CCNA2', 'CCNA2_POLI' ]].X).statistic)
df.columns = ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3', 'BATF2_CCNA2', 'CCNA2_BATF2','TAF1_CCNA2', 'CCNA2_TAF1', 'POLI_CCNA2', 'CCNA2_POLI' ]
df.index = ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3', 'BATF2_CCNA2', 'CCNA2_BATF2','TAF1_CCNA2', 'CCNA2_TAF1', 'POLI_CCNA2', 'CCNA2_POLI' ]
df

In [ ]:
df = pd.DataFrame(st.spearmanr(grn_adata[grn_adata.obs['grn'] == 2.0, ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3', 'BATF2_CCNA2', 'CCNA2_BATF2','TAF1_CCNA2', 'CCNA2_TAF1' , 'POLI_CCNA2', 'CCNA2_POLI' ]].X).statistic)
df.columns = ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3', 'BATF2_CCNA2', 'CCNA2_BATF2','TAF1_CCNA2', 'CCNA2_TAF1', 'POLI_CCNA2', 'CCNA2_POLI' ]
df.index = ['CCNA2_BBX', 'BBX_CCNA2', 'KAT7_CCNA2', 'CCNA2_KAT7', 'TBP_CCNA2', 'CCNA2_TBP', 'BCAS3_CCNA2', 'CCNA2_BCAS3', 'BATF2_CCNA2', 'CCNA2_BATF2','TAF1_CCNA2', 'CCNA2_TAF1', 'POLI_CCNA2', 'CCNA2_POLI']
df

In [ ]:
from src.evaluation.compute_metrics import  get_top_edges_per_cell, get_top_edges_per_cell_per_cluster, downstream_recipe

In [ ]:
grn_adata.obs['grn'] = pd.Categorical(adata.obs['grn'])

In [ ]:
intersection_genes = grn_adata.var.index.intersection(pairs_full)
intersection_indices = [pairs_full.index(gene) for gene in intersection_genes]

In [ ]:
np.array(pairs_full)[intersection_indices]

In [ ]:
aa = np.multiply(grn_adata.X[:, grn_adata.var.index.isin(grn_adata.var.index.intersection(pairs_full))], mask_full[:, intersection_indices])
aa = ad.AnnData(aa)

In [ ]:
grn_adata = downstream_recipe(grn_adata)
grn_adata.var.columns = ['source', 'target', 'n_cells']

In [ ]:
top  = top.loc[:,['source', 'target']].groupby(['source', 'target']).value_counts().reset_index()

In [ ]:
top.sort_values('count', ascending=False)[0:50]

In [ ]:
nets[1]

In [ ]:
top.sort_values('count', ascending=False)
top.merge(nets[0], left_on = ['source', 'target'], right_on=['source', 'target']).sort_values('count', ascending=False)

In [ ]:
top.sort_values('count', ascending=False)
top.merge(nets[0], left_on = ['source', 'target'], right_on=['source', 'target'])

In [ ]:
['APP', 'ETS2', 'KAT2A', 'KRT36', 'POLI', 'SLC3A1', 'TRPV4']
top.sort_values('count', ascending=False)[0:50]

In [ ]:
top.merge(common[0], left_on = ['source', 'target'], right_on=['source', 'target'])

In [ ]:
top.merge(common[0], left_on = ['source', 'target'], right_on=['source', 'target'])

In [ ]:
adata.obs.grn = pd.Categorical(adata.obs.grn)

In [ ]:
sc.pl.violin(adata, keys = ['POT1', 'LRMDA'], groupby='grn')

In [ ]:
top.merge(nets[0])

In [ ]:
aa.obs['grn'] = pd.Categorical(adata.obs['grn'])

In [ ]:
grn_adata2 = aa[:, aa.var_names.isin(top['edgenames'])]

In [ ]:
grn_adata2  = downstream_recipe(grn_adata2)

In [ ]:
sc.pl.umap(grn_adata2, color = [ 'grn'])

In [ ]:
mynets = [(1.0, nets[0]), (2.0, nets[1])]

In [ ]:
get_top_edges_per_cell_per_cluster(aa,  mynets, cluster_var = 'grn', top_edges= 20000)

In [ ]:
pd.set_option('display.float_format', lambda x: '%8f' % x)

In [ ]:
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

suba = grn_adata[:, grn_adata.var.source=='MTA1']
suba = grn_adata[:, grn_adata.var.source=='GATA3']

corr_matrix = np.corrcoef(suba.X.T)

corr_dist = 1 - corr_matrix
dist_linkage = hierarchy.average(corr_dist)


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 15))

# We convert the correlation matrix to a distance matrix before performing
# hierarchical clustering using Ward's linkage.

dendro = hierarchy.dendrogram(
    dist_linkage, labels=suba.var.target.to_list(), ax=ax1, leaf_rotation=90
)
dendro_idx = np.arange(0, len(dendro["ivl"]))

ax2.imshow(corr_matrix[dendro["leaves"], :][:, dendro["leaves"]])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(dendro["ivl"], rotation="vertical")
ax2.set_yticklabels(dendro["ivl"])
_ = fig.tight_layout()
    
clusters = hierarchy.fcluster(dl, t=2, criterion='distance')
cluster_to_genes = {}
for gene_name, cluster_id in zip(gene_names1, clusters):
    if cluster_id not in cluster_to_genes:
        cluster_to_genes[cluster_id] = []
    cluster_to_genes[cluster_id].append(gene_name)
